# Example: Potjans & Diesmann microcircuit  2014

This example notebook submits a job to a chosen HPC backend of EBRAINS/HBP (**JUSUF**), loads a **Singularity** container with a complete **hybridLFPy** installation and executes the hybridLFPy example file ``example_microcircuit.py`` (https://github.com/INM-6/LFPy/blob/nest3/examples/example_microcircuit.py) in parallel on the backend.

For more in-depth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

## Prepare connection to backend

In [ ]:
# use the pyunicore library
!pip install pyunicore --upgrade

In [ ]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame, clear_output

In [ ]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [ ]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

In [ ]:
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [ ]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPydebian/main/mpich.Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download hybridLFPy container
- download simulation files from GitHub (https://github.com/INM-6/examples/example_microcircuit.py and additional input files)
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation via the batch job queue
- download tar file with simulation output

### Notes

Singularity exposes the host environment to the running container (i.e., ``$PATH``, ``$PYTHONPATH``, ``$HOME`` etc.). 
The container includes Python 3.8.5, and may find pip-installed software in your host's ``$HOME/.local/lib/python3.8/site-packages`` directory if it exist. 

If you encounter problems, try including ``unset PYTHONPATH`` in your ``simulation_job["Executable"]`` part below.

In [ ]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = []

In [ ]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    #"Nodes": "2",
    "CPUs": "512",
    "Runtime": "1200",
}

In [ ]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools git

git clone --branch nest3 https://github.com/INM-6/hybridLFPy.git
cd hybridLFPy/examples
sib download --recipe-name hybridlfpy
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [ ]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node(s)
# unset SCRATCH  # we're anyway working on $SCRATCH - write sim output to simulation script folder
export SINGULARITYENV_SCRATCH=  # make container unaware of $SCRATCH area as we're already there - write sim output to simulation script folder
export SINGULARITYENV_PYTHONPATH=/opt/nest/lib/python3.8/site-packages
cd hybridLFPy/examples  # go to examples
singularity exec hybridlfpy.sif nrnivmodl  # compile NEURON NMODL (.mod) files
srun singularity exec hybridlfpy.sif python3 -u example_microcircuit.py  # execute simulation
"""

In [ ]:
# commands run after job is done
simulation_job["User postcommand"] = "" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [ ]:
simulation_job

In [ ]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [ ]:
job.working_dir

In [ ]:
# wait while job is running and track progress
while job.is_running():
    clear_output()
    # STDERR output (if any)
    try:
        stderr = job.working_dir.stat('stderr')
        #pprint(stderr.raw().readlines()[:5])
        pprint(stderr.raw().readlines()[-5:])
    except:
        print('no stderr file to read yet')
    # STDOUT output
    try:
        stdout = job.working_dir.stat('stdout')
        #pprint(stdout.raw().readlines()[:10])
        pprint(stdout.raw().readlines()[-10:])
    except:
        print('no stdout file to read yet')
    sleep(10)

In [ ]:
job.working_dir.listdir().keys()

In [ ]:
# download simulation output
job.working_dir.stat('hybridLFPy/examples/simulation_output_example_microcircuit.tar'
                     ).download('simulation_output_example_microcircuit.tar')

In [ ]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output

In [ ]:
# untar output to folder example_network_output
!tar -xf simulation_output_example_microcircuit.tar

In [ ]:
# quick look at e.g., the extracellular potential and spike raster plot
#IFrame("./example_network_output/extracellular_potential.pdf", width=800, height=600)

In [ ]:
#IFrame("./example_network_output/spike_raster.pdf", width=800, height=600)